In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import nucleus

client = nucleus.NucleusClient(os.environ["NUCLEUS_API_KEY"], endpoint="http://localhost:3000/v1/nucleus")

In [2]:
from nucleus.metrics import Metric, MetricResult
from nucleus.annotation import AnnotationList
from nucleus.prediction import PredictionList


class MyMetric(Metric):
    def __call__(
        self, annotations: AnnotationList, predictions: PredictionList
    ) -> MetricResult:
        value = (len(annotations) - len(predictions)) ** 2
        weight = len(annotations)
        return MetricResult(value, weight)

eval_func = client.modelci.upload_eval_function(MyMetric(), "my_metric")

-------------------------------------  -------------------  ------------------  ------------------  ------------------
Fixture name                           PolygonIOU           PolygonPrecision    PolygonRecall       Your custom metric
many_images_many_fps_box.json          0.4255035644058599   0.5021834061135371  0.9663865546218487  20.529411764705884
many_images_mediocre_matches_box.json  0.4699543977744903   0.6                 0.5                 0.5
single_image_many_fps_box.json         0.29066466713084466  0.3333333333333333  0.9999999999999998  4.0
single_image_missing_gt_box.json       0.0                  0.0                 0.0                 0.0
single_image_missing_preds_box.json    0.0                  0.0                 0.0                 1.0
single_image_perfect_match_box.json    1.0                  1.0                 1.0                 0.0
-------------------------------------  -------------------  ------------------  ------------------  ------------------


In [ ]:
from nucleus import BoxAnnotation
from nucleus.metrics import MetricResult, PolygonMetric
from nucleus.metrics.polygon_utils import BoxOrPolygonAnnotation, BoxOrPolygonPrediction, iou_assignments

class MyPolygonMetric(PolygonMetric):
    def eval(
        self,
        annotations: List[BoxOrPolygonAnnotation],
        predictions: List[BoxOrPolygonPrediction],
    ) -> MetricResult:
        iou_assigns = iou_assignments(
            annotations, predictions, self.iou_threshold
        )
        value = 0.0 # TODO
        weight = 0.0 # TODO
        return MetricResult(value, weight)


In [3]:
client.modelci.eval_functions.private_functions

{'my_metric': <EvalFunction: name=my_metric, id=ef_c7qexkyvshg2w8199p10>}

In [ ]:
client.modelci.delete_eval_function(eval_func.id)

In [ ]:
# TODO
# If an eval fn is deleted and results in unit test with 0 metrics, maybe warn / error?